In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")

In [78]:
from data import MNISTDatamodule
from model import ImageClassifier
from torch import nn
from torchvision import transforms as T
import pytorch_lightning as pl
from architectures.resnet import ResNet

In [80]:
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]
mean = [0.1307]
std = [0.3081]
transform = T.Compose(
    [
        T.ToTensor(),
        T.Normalize(mean=mean, std=std),
        # nn.Flatten(start_dim=0, end_dim=-1)
    ]
)

In [81]:
dm = MNISTDatamodule(
    data_dir="../data",
    transform = transform,
)

In [82]:
dm.download_data()
dm.setup()

In [83]:
X, y = next(iter(dm.train_dataloader()))

In [84]:
from architectures.resnet import ResNet, BasicBlock, BottleneckBlock

In [85]:
X.shape

torch.Size([64, 1, 28, 28])

In [99]:
from torchinfo import summary
import torch

In [101]:
summary(resnet, input_size=X.shape)

/home/shate/.local/lib/python3.11/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/shate/.local/lib/python3.11/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [64, 256, 1, 1]           --
├─Conv2d: 1-1                                      [64, 64, 13, 13]          640
├─MaxPool2d: 1-2                                   [64, 64, 7, 7]            --
├─BasicResNetCore: 1-3                             [64, 256, 2, 2]           --
│    └─Sequential: 2-1                             [64, 256, 2, 2]           --
│    │    └─Sequential: 3-1                        [64, 64, 7, 7]            74,112
│    │    └─Sequential: 3-2                        [64, 128, 4, 4]           230,528
│    │    └─Sequential: 3-3                        [64, 256, 2, 2]           919,808
├─AdaptiveAvgPool2d: 1-4                           [64, 256, 1, 1]           --
Total params: 1,225,088
Trainable params: 1,225,088
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 709.05
Input size (MB): 0.20
Forward/backward pass size (MB): 21.40
P

In [102]:
resnet = ResNet(
    in_channels=1,
    stem_channels=64,
    stem_kernel_size=3,
    block_type="bottleneck",
    stages_n_blocks=[1, 1, 1],
    pool_kernel_size=1
)

In [104]:
resnet(X).shape

torch.Size([64, 1024, 1, 1])